In [1]:
import os 
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_num_class: int
    params_device: str
    params_learning_rate: float

In [4]:
from src.CNNClassifier.constants import * 
from src.CNNClassifier.utils.common import read_yaml, create_directories

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import time

from tqdm import tqdm 

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model =self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.root_dir, "CT-Scan")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_num_class = params.CLASSES,
            params_device = params.DEVICE,
            params_learning_rate = params.LEARNING_RATE
        )

        return training_config

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def get_base_model(self):
        # self.model = models.vgg16(pretrained=True)
        # self.model.classifier[6] = torch.nn.Linear(self.model.classifier[6].in_features, self.config.params_num_class)

        self.device = self.config.params_device
        self.model = torch.load(self.config.updated_base_model_path)
        print(self.model)
        self.model = self.model.to(self.device)

    def train_valid_genertor(self):
        data_transforms = {
            'train': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'test': transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }   

        train_dataset = datasets.ImageFolder(root=Path(f'{self.config.training_data}/train'), transform=data_transforms['train'])
        val_dataset = datasets.ImageFolder(root=Path(f'{self.config.training_data}/test'), transform=data_transforms['test'])
        
        self.train_loader = DataLoader(train_dataset, batch_size=self.config.params_batch_size, shuffle=True, num_workers=4)
        self.val_loader = DataLoader(val_dataset, batch_size=self.config.params_batch_size, shuffle=False, num_workers=4)
    
    def train(self):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.classifier.parameters(), lr=self.config.params_learning_rate)

        best_acc = 0.0  

        for epoch in tqdm(range(self.config.params_epochs), desc="Training: "):
            for phase in ['train', 'val']:
                self.model.train() if phase == 'train' else self.model.eval()  
                data_loader = self.train_loader if phase == 'train' else self.val_loader

                running_loss, running_corrects = 0.0, 0

                for inputs, labels in data_loader:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = self.model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels)

                epoch_loss = running_loss / len(data_loader.dataset)
                epoch_acc = running_corrects.double() / len(data_loader.dataset)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(self.model.state_dict(), self.config.trained_model_path)
        

In [8]:
try: 
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_genertor()
    training.train()
    
except Exception as e:
    raise e

[2024-10-24 18:09:03,154: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-24 18:09:03,158: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-24 18:09:03,160: INFO: common: created directory at: artifacts]
[2024-10-24 18:09:03,162: INFO: common: created directory at: artifacts\training]
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1,

Training:   0%|          | 0/1 [00:00<?, ?it/s]

train Loss: 0.9274 Acc: 0.5759
val Loss: 0.7638 Acc: 0.6825


Training: 100%|██████████| 1/1 [00:16<00:00, 16.39s/it]


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [9]:
data_dir = 'artifacts\data_ingestion\CT-Scan'  # Replace with the path to your dataset
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=f'{data_dir}/test', transform=data_transforms['test'])

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [11]:
model = models.vgg16(pretrained=True)

# Freeze all layers
for param in model.features.parameters():
    param.requires_grad = False

# Replace the final layer to match the 2-class problem
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 4)
model = model.to(device)

d:\Anaconda\envs\mlops\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\mlops\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [14]:
# Training function with best model saving and returning
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    best_acc = 0.0
    best_model_weights = None  # Store best weights

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Training mode
                data_loader = train_loader
            else:
                model.eval()  # Evaluation mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over batches
            for inputs, labels in data_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)  # Model output
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize during training
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_weights = model.state_dict()

    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights and save to disk
    model.load_state_dict(best_model_weights)
    torch.save(model.state_dict(), 'best_model.pth')  # Save the best model

    return model 

Epoch 1/10
----------
train Loss: 1.2124 Acc: 0.5367
val Loss: 0.8403 Acc: 0.5397
Epoch 2/10
----------
train Loss: 0.8471 Acc: 0.6183
val Loss: 1.2313 Acc: 0.4825
Epoch 3/10
----------
train Loss: 0.6900 Acc: 0.7194
val Loss: 0.7431 Acc: 0.6762
Epoch 4/10
----------
train Loss: 0.3735 Acc: 0.8891
val Loss: 0.8398 Acc: 0.7143
Epoch 5/10
----------
train Loss: 0.3829 Acc: 0.9054
val Loss: 2.9060 Acc: 0.4063
Epoch 6/10
----------
train Loss: 0.3346 Acc: 0.9184
val Loss: 2.0034 Acc: 0.6095
Epoch 7/10
----------
train Loss: 0.3423 Acc: 0.9070
val Loss: 1.8627 Acc: 0.6698
Epoch 8/10
----------
train Loss: 0.3777 Acc: 0.9315
val Loss: 4.4125 Acc: 0.5968
Epoch 9/10
----------
train Loss: 0.4414 Acc: 0.9282
val Loss: 4.0736 Acc: 0.6698
Epoch 10/10
----------
train Loss: 0.5912 Acc: 0.9282
val Loss: 10.7960 Acc: 0.5841
Best val Acc: 0.7143


In [23]:
# Train the model and get the trained version
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

print("Training complete. Best model saved as 'best_model.pth'.")

torch.Size([32])